# Pattern Recognition Work
                                                                                         
- Implement the C4.5 Decision Tree algorithm to classify the database provided
- In the C4.5 algorithm, you must use entropy (information gain) as a criterion for choosing the nodes
- Pruning is not necessary
- Randomly divide data between training and testing.
- You must choose the attributes that you find most convenient
- At least 4 attributes must be used.


                                                                                    Alanna Maria Machado Alves Paiva
                                                                                           Ananda Káren Barros Nobre

#### Imports needed

In [1]:
import pandas as pd
import numpy as np
import math
import operator
from collections import Counter

#### Load database

In [2]:
df = pd.read_csv('wine.csv', header=None)
columns = ['Classe', 'Alcool', 'Acido malico', 'Cinza', 'Alcalinidade da cinza','Magnesio',
           'Fenois totais', 'Flavonoides','Fenois nao flavonoides','Proantocianidinas',
           'Intensidade da cor','Matiz','OD280/OD315 de vinhos diluidos','Prolina']
df.columns = columns
df.head()

,Classe,Alcool,Acido malico,Cinza,Alcalinidade da cinza,Magnesio,Fenois totais,Flavonoides,Fenois nao flavonoides,Proantocianidinas,Intensidade da cor,Matiz,OD280/OD315 de vinhos diluidos,Prolina
0,1,14.23,1.71,2.43,15.6,127,2.80,3.06,0.28,2.29,5.64,1.04,3.92,1065
1,1,13.20,1.78,2.14,11.2,100,2.65,2.76,0.26,1.28,4.38,1.05,3.40,1050
2,1,13.16,2.36,2.67,18.6,101,2.80,3.24,0.30,2.81,5.68,1.03,3.17,1185
3,1,14.37,1.95,2.50,16.8,113,3.85,3.49,0.24,2.18,7.80,0.86,3.45,1480
4,1,13.24,2.59,2.87,21.0,118,2.80,2.69,0.39,1.82,4.32,1.04,2.93,735


## 2. Classifiers 
#### 2.1 Implement the classifier: C4.5 (Decision Tree)
- Should use entropy (information gain) as a criterion for choosing nodes.
- You do not need to perform pruning.
- You should choose the attributes you find most convenient.
- You should use at least 4 attributes.
- Do not use ready-made Matlab/Python functions for C4.5

#### Attribute choices
As stated in the work, the columns given already have the attributes to be chosen, so I chose the first four, which are:

* Alcohol
* Malic Acid 
* Gray
* Ash Alkalinity

There is no problem in choosing other attributes.

In [3]:
# Separa os 4 atributos
atributos = pd.DataFrame({})
atributos['Classe']  = df['Classe'] 
atributos[df.iloc[:,1].name]  = df.iloc[:,1]
atributos[df.iloc[:,2].name]  = df.iloc[:,2]
atributos[df.iloc[:,3].name]  = df.iloc[:,3]
atributos[df.iloc[:,4].name]  = df.iloc[:,4]
atributos


# We will remove a class 
atributos.drop(atributos.query('Classe==3').index,inplace=True)

# Sorteia a nova tabela por linha
atributos = atributos.sample(frac=1).reset_index(drop=True)
display(atributos)

,Classe,Alcool,Acido malico,Cinza,Alcalinidade da cinza
0,1,13.05,1.65,2.55,18.0
1,1,13.24,2.59,2.87,21.0
2,2,12.25,1.73,2.12,19.0
3,2,11.84,2.89,2.23,18.0
4,1,14.10,2.02,2.40,18.8
...,...,...,...,...,...
125,1,14.37,1.95,2.50,16.8
126,1,14.20,1.76,2.45,15.2
127,2,13.03,0.90,1.71,16.0
128,1,14.06,1.63,2.28,16.0


#### Auxiliary Functions

In [4]:
# Função que calcula a entropia dos dados
def entropy(df_label):
    classes,class_counts = np.unique(df_label,return_counts = True)
    entropy_value = np.sum([(-class_counts[i]/np.sum(class_counts))*np.log2(class_counts[i]/np.sum(class_counts)) 
    for i in range(len(classes))])
    return entropy_value

# Função que calcula a acurácia 
def accuracy(original, predicted):
    correct = 0
    for i in range(len(original)):
        if original[i] == predicted[i]:
            correct += 1
    return correct / float(len(original)) * 100.0

# Função que verifica e retorna a classe mais comum 
def most_common_label(y):
    most_common = Counter(y).most_common(1)
    return most_common[0][0]

# Função que retorna os índices dos filhos
def split(X_column, split_thresh):
    column = np.array(X_column)
    left_idxs = np.argwhere(column <= split_thresh).flatten()
    right_idxs = np.argwhere(column > split_thresh).flatten()
    return left_idxs, right_idxs

#### Implementing Decision Trees (C4.5)

In [5]:
class Node:
    def __init__(
    self, feature=None, threshold=None, left=None, right=None, *, value=None
    ):
        self.feature = feature
        self.threshold = threshold
        self.left = left
        self.right = right
        self.value = value

    def is_leaf_node(self):
        return self.value is not None


class DecisionTree:
    def __init__(self, min_samples_split=2, max_depth=100):
        self.min_samples_split = min_samples_split
        self.max_depth = max_depth
        self.root = None

    def fit(self, X_train, y_train):
        self.root = self.grow(X_train, y_train)

    def predict(self, X_train):
        return np.array([self.traverse_tree(x, self.root) for x in X_train])

    def grow(self, X_train, y_train, depth=0):
            n_samples, n_features = X_train.shape
            n_labels = len(np.unique(y_train))
            
            # Critérios de parada
            if (
                depth >= self.max_depth
                or n_labels == 1
                or n_samples < self.min_samples_split
            ):
                leaf_value = most_common_label(y_train)
                return Node(value=leaf_value)

            # Calcula o melhor limiar para cada atributo
            gains, idx = list(), -1
            
            for i in X_train.T: 
                idx += 1
                gains.append(self.chose_threshold(X_train[:, idx], y_train, idx))

            # Armazena o melhor limiar e o atributo a qual ele pertence 
            gains = sorted(gains)
            best_thresh, best_feat = gains[0][1], gains[0][2]
    
            # Divide os dados pelo limiar e cria nós filhos resultantes da divisão
            left_idxs, right_idxs = split(X_train[:, best_feat], best_thresh)
            left = self.grow(X_train[left_idxs, :], y_train[left_idxs], depth + 1)
            right = self.grow(X_train[right_idxs, :], y_train[right_idxs], depth + 1)
            return Node(best_feat, best_thresh, left, right)
    
    def chose_threshold(self, atributo, classe, idx):
        label_ord, val_ord = np.array(classe), np.array(atributo)
        
        N = len(label_ord)
        lbl_ant = label_ord[0] 
        limiares = list()

        # Calcula os limiares
        for i in range(N-1):
            lbl = label_ord[i+1]
            if(lbl != lbl_ant):
                limiares.append((val_ord[i+1] + val_ord[i]) / 2)

            lbl_ant = lbl

        # Divide dados em dois grupos, atraves do valor do limiar
        Nlimiares = len(limiares)
        mean_entropy = []
        
        # Cada limiar divide os dados em dois grupos
        for i in range(Nlimiares):
            limiar = limiares[i]

            gr1, gr2, grp_entropy1, grp_entropy2 = list(), list(), list(), list()
            for j in range(N):
                if (val_ord[j] < limiar ):
                    gr1.append(label_ord[j])
                else: 
                    gr2.append(label_ord[j])
            # Calcula e armazena a entropia dos grupos
            grp_entropy1.append(entropy(gr1)), grp_entropy2.append(entropy(gr2))
            # Calcula e entropia média dos grupos
            mean_entropy.append([(a + b) / 2 for a, b in zip(grp_entropy1, grp_entropy2)])
        # Armazena a menor entropia e o seu indice
        index, value = min(enumerate(mean_entropy), key=operator.itemgetter(1))
        return value, limiares[index], idx


    def traverse_tree(self, x, node):
        if node.is_leaf_node():
            return node.value
        if x[node.feature] <= node.threshold:
            return self.traverse_tree(x, node.left)
        return self.traverse_tree(x, node.right)

## 3)The C4.5 descision tree algorithm (Make 10 achievements )

#### a) Realization:
- Randomly divide data between training and testing. (Ex: 80% for training, 20% for testing)
- Build the model with the training data
- Check the accuracy (hit rate) of the model (test). 
b) Statistics:
- Calculate the average hit rate of the 10 realizations.

In [6]:
from sklearn.model_selection import train_test_split
for i in range(10):

    X, y = np.array(atributos.drop('Classe', axis=1)), np.array(atributos['Classe']) 

    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.2, random_state=1234)

    clf = DecisionTree(max_depth=10)
    clf.fit(X_train, y_train)

    y_pred = clf.predict(X_test)
    acc = accuracy(y_test, y_pred)

    scores = []
    scores.append(acc)

    print(f'Accuracy: %.2f%%' % acc)

Accuracy: 96.15%
Accuracy: 96.15%
Accuracy: 96.15%
Accuracy: 96.15%
Accuracy: 96.15%
Accuracy: 96.15%
Accuracy: 96.15%
Accuracy: 96.15%
Accuracy: 96.15%
Accuracy: 96.15%


#### b) Statistics:
- Calculate the average hit rate of the 10 runs.

In [7]:
mean = (sum(scores)/float(len(scores)))
print(f'Mean Accuracy: %.2f%%' % mean)

Mean Accuracy: 96.15%
